# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Importing-data" data-toc-modified-id="Importing-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing data</a></div><div class="lev1 toc-item"><a href="#Helper-functions" data-toc-modified-id="Helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Helper functions</a></div><div class="lev1 toc-item"><a href="#Comorbidities" data-toc-modified-id="Comorbidities-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Comorbidities</a></div><div class="lev1 toc-item"><a href="#Clusters-(diseases-+-demographics)" data-toc-modified-id="Clusters-(diseases-+-demographics)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clusters (diseases + demographics)</a></div>

Here's my attempt at straightening all this old code out

In [1]:
import pandas as pd
import itertools
import numpy as np

from ast import literal_eval

from helper_functions import *
#     functions contained:
#     prop_error(n, p, alpha = 0.05)
#     prop_error_pct(n, p, alpha = 0.05)
#     epi_rows(df, col, year = None)
#     age_bin(age, labels, bins)
#     comorbidity_indicator(data, cc)
#     cost_center_indicator(data, cc)
#     prevalence(data, indicator)
#     age_adjusted_frequency(data, indicator=None, comorbidities=None, weights = pd.Series([]), yr = None)

# Importing data

In [2]:
df_merged = pd.read_csv('data/YOUR DATA HERE')
df_merged.classes = df_merged.classes.apply(lambda x: literal_eval(x))
df_merged.agg_indices = df_merged.agg_indices.fillna('None').apply(lambda x: literal_eval(x))

df_merged.sex.fillna('Not provided')
pop_N = df_merged.shape[0] # grab total number of person-years

# IMPORT DESCRIPTION AND CONDITION INDICES
cond_dict = pd.read_excel('data/disease_dictionary.xls').short.to_dict()
desc_dict = pd.read_excel('data/disease_dictionary.xls').long.to_dict()

# We will be grouping by pairs of comorbid diseases.
disease_dyads = [i for i in itertools.combinations(range(len(desc_dict)), 2)]

In [4]:
# age filters
df_merged['age_gt_65'] = df_merged.age > 65 

# age bins
bins = [0,18,35,50,65,150]
labels = ['0-18','18-35','35-50','50-65','65+']
df_merged['age_bin'] = pd.cut(df_merged.age, bins=bins, labels=labels)

sexes = ['F','M']

In [5]:
# convert comorbidity list to boolean columns
## comorbidity_indicator()
for i in range(len(desc_dict)):
    df_merged[desc_dict[i]] = comorbidity_indicator(df_merged,[str(i)])

# Helper functions

In [6]:
def segment_stats(data):
    row_dict = {}
    row_dict['avg_cost'] = data.sum_costs.mean()
    row_dict['std_cost'] = data.sum_costs.std()
    row_dict['num_person_years'] = data.shape[0]
    row_dict['avg_num_com'] = data.no_comorbidities.mean()
    row_dict['std_num_com'] = data.no_comorbidities.std()
    
    return row_dict

# for i in c_dyads:
#     temp = df_merged[(df_merged[i[0]]) & (df_merged[i[1]])]
#     c_dyad_df = c_dyad_df.append(pd.Series(cluster_stats(temp), name=i)

In [7]:
## Observed / Expected!!
def oe(data, a, b):
    freq_a = data[data[desc_dict[a]]].shape[0] / pop_N
    freq_b = data[data[desc_dict[b]]].shape[0] / pop_N
    expected = freq_a * freq_b
    observed = data[(data[desc_dict[a]]) & (data[desc_dict[b]])].shape[0] / pop_N
    
    return {'observed': observed, 'expected': expected}


# Comorbidities

In [9]:
# comorbid disease dyad stats
d_dyad_df = pd.DataFrame()

for i in disease_dyads:
    temp = df_merged[(df_merged[desc_dict[i[0]]]) & (df_merged[desc_dict[i[1]]])]
    d_dyad_df = d_dyad_df.append(pd.Series(segment_stats(temp), name=i))

In [11]:
oe_df = pd.DataFrame()

for i in d_dyad_df.index.tolist():
    oe_df = oe_df.append(pd.Series(oe(df_merged, i[0], i[1]), name = i))
    
d_dyad_df = pd.concat([d_dyad_df, oe_df], axis = 1)

In [ ]:
d_dyad_df['O/E'] = d_dyad_df.observed / d_dyad_df.expected
d_dyad_df['frequency'] = d_dyad_df.num_person_years / df_merged.shape[0]
d_dyad_df['names'] = d_dyad_df.index.map(lambda x: [desc_dict[i] for i in x])

In [ ]:
d_dyad_df.to_csv('disease_dyads_df.csv')

# Segments (diseases + demographics)

In [ ]:
# diseases + male or female
b = [sexes, disease_dyads]

combos = list(itertools.product(*b))


ds_dyad_df = pd.DataFrame()

for i in combos:
    temp = df_merged[(df_merged.sex == i[0]) & (df_merged[desc_dict[i[1][0]]]) & (df_merged[desc_dict[i[1][1]]])]
    ds_dyad_df = ds_dyad_df.append(pd.Series(segment_stats(temp), name=i))

# [over_65,sexes,diseases,cost_centers]

ds_dyad_df.to_csv('ds_dyad_df.csv')

In [ ]:
# diseases + male or female + age_bin
b = [sexes, age_bins, disease_dyads]

combos = list(itertools.product(*b))

das_dyad_df = pd.DataFrame()

for i in combos:
    temp = df_merged[(df_merged.sex == i[0]) & \
                     (df_merged.age_bin == i[1]) & \
                     (df_merged[desc_dict[i[2][0]]]) & \
                     (df_merged[desc_dict[i[2][1]]])]
    das_dyad_df = das_dyad_df.append(pd.Series(segment_stats(temp), name=i))

# [over_65,sexes,diseases,cost_centers]

das_dyad_df.to_csv('das_dyad_df.csv')

In [ ]:
das_dyad_df = pd.read_csv("das_dyad_df.csv", index_col=0 , converters={"index": lambda x: x.strip("[]").split(", ")})

In [ ]:
das_dyad_df['segment_type'] = das_dyad_df.index
das_dyad_df['segment_type'] = das_dyad_df['segment_type'].apply(literal_eval)

In [ ]:
das_dyad_df['segment_name_long'] = das_dyad_df['segment_type'].apply(lambda x: [x[0], x[1], [desc_dict[i] for i in x[2]]])
das_dyad_df['segment_name_short'] = das_dyad_df['segment_type'].apply(lambda x: [x[0], x[1], [cond_dict[i] for i in x[2]]])
das_dyad_df['total_cost'] = das_dyad_df.avg_cost * das_dyad_df.num_person_years
das_dyad_df['age_group'] = das_dyad_df['segment_type'].apply(lambda x: x[1])
das_dyad_df['sex'] = das_dyad_df['segment_type'].apply(lambda x: x[0])
das_dyad_df[['first_condition', 'second_condition']] = das_dyad_df['segment_name_long'].apply(lambda x: pd.Series(x[2]).transpose())

In [ ]:
#Filter clusters/segments in a bunch of different ways
das_dyad_df[das_dyad_df.num_person_years > 10].sort_values(by='avg_cost', ascending=False).to_csv('das_dyads_cost_gt10.csv')
das_dyad_df[das_dyad_df.num_person_years > 100].sort_values(by='avg_cost', ascending=False).to_csv('das_dyads_cost_gt100.csv')
das_dyad_df[das_dyad_df.num_person_years > 1000].sort_values(by='avg_cost', ascending=False).to_csv('das_dyads_cost_gt1000.csv')
das_dyad_df.sort_values(by='num_person_years', ascending=False).to_csv('das_dyads_freq_all.csv')
das_dyad_df.sort_values(by='total_cost', ascending=False).to_csv('das_dyads_totalcost_all.csv')

In [ ]:
#Filter by specific demographics
das_dyad_df[(das_dyad_df.sex == 'M') & (das_dyad_df.age_group == '35-50')].sort_values(by='total_cost', ascending=False).to_csv('das_dyads_totalcost_all_M_35-50.csv')
das_dyad_df[(das_dyad_df.sex == 'F') & (das_dyad_df.age_group == '35-50')].sort_values(by='total_cost', ascending=False).to_csv('das_dyads_totalcost_all_F_35-50.csv')
das_dyad_df[(das_dyad_df.sex == 'M') & (das_dyad_df.age_group == '35-50')].sort_values(by='avg_cost', ascending=False).to_csv('das_dyads_avgcost_all_F_35-50.csv')
das_dyad_df[(das_dyad_df.sex == 'F') & (das_dyad_df.age_group == '35-50')].sort_values(by='avg_cost', ascending=False).to_csv('das_dyads_avgcost_all_F_35-50.csv')

In [ ]:
das_dyad_df.to_csv("df_table1.csv")